# EquiHealth: Health Data Collection

This notebook will be used to **implement the collection of health-related datasets** for the EquiHealth project. 
The goal is to gather structured data that can later be used to analyze healthcare inequalities across districts and support the hospital complaint portal.

---

## Objectives

- Identify sources for multiple health datasets, including hospital facilities and population statistics.
- Plan the steps required to clean, standardize, and store data for future analysis and visualization.
- Outline the structure of processed datasets for dashboards and reporting.

---

## Planned Target Datasets

1. **Hospital / Healthcare Facility Data**
   - Sources: National Health Portal Hospital Directory, Rural Health Statistics (RHS), HMIS portal
   - Planned fields: Name, Type, Ownership, Address, District, State, Pincode, Contact info, Services, Beds, Geo-coordinates, Last updated, Source URL

2. **Population Data**
   - Sources: Census of India, district/state population projections
   - Planned fields: District/City name and code, Total population, Age groups, Urban/Rural split, Population density, Source URL, Year

3. **Other Health Indicators (Future / Optional)**
   - Staff availability, equipment, sanctioned vs functional beds, program coverage
   - Sources: HMIS, Ministry of Health reports, government portals

---

## Planned Data Fields

- **Facility Table (to be created):**  
  `facility_id`, `name`, `type`, `ownership`, `address`, `district`, `state`, `pincode`, `latitude`, `longitude`, `contact`, `services`, `beds`, `last_updated`, `source_url`

- **Population Table (to be created):**  
  `district_code`, `district_name`, `state_name`, `total_population`, `age_groups`, `urban_rural`, `source_url`, `year`

- **Optional Metrics / Indicators (to be considered later):**  
  Staff counts, ICU beds, equipment availability, program coverage

---

## Planned Workflow

1. Identify official sources and APIs for hospital, population, and other health datasets.
2. Plan how to download or scrape raw datasets.
3. Plan data cleaning and normalization steps (facility names, addresses, district codes).
4. Plan merging and transformation into master tables for analysis.
5. Plan saving processed data in `data/processed/` for visualization and portal integration.


## Scraping Hospital Data from Jeevandayee Portal (Maharashtra)

The [**Jeevandayee Portal**](https://www.jeevandayee.gov.in/MJPJAY/FrontServlet?requestType=PublicViewsRH&actionVal=ViewBedInfoForDisease&City=x%20&Disease=-1&DataFlag=true&DfltHospList=Reports) is the official website for the **Mahatma Jyotiba Phule Jan Arogya Yojana (MJPJAY)**.  
It contains information about empaneled hospitals in **all districts of Maharashtra**.

### Why This Data?
For the **EquiHealth project**, we are collecting hospital data for the whole of India.  
From this portal, we will get hospital data for Maharashtra, including:

- Hospital name  
- District  
- Taluka (sub-district)  
- Address  
- Pincode  
- MCO contact number  
- Total beds  
- Hospital type (Government / Corporate)  
- Government sub-type (for government hospitals)

This data will later be combined with other states’ data to create a **national hospital dataset**.

### Planned Scraping Method
The portal is **interactive**:

- Hospital lists appear after selecting a district and clicking search.  
- Additional details are in separate tables or links.  
- No direct download or API is available.  

We plan to use **Selenium WebDriver** to:

1. Open the portal in a browser.  
2. Scrape **basic hospital info** for all districts.  
3. Enrich data with **detailed hospital info**, including taluka, address, pincode, and MCO contact.  
4. Identify **hospital type** and **Government sub-type**.  
5. Scrape all **hospital service categories**.  
6. Build the **hospital-category mapping table**.

### Output
The planned datasets:

1. `hospital.csv` → Detailed hospital information.  
2. `category.csv` → Hospital service categories.  
3. `hospital_category.csv` → Maps hospitals to categories.

These datasets will form the foundation for **EquiHealth’s analysis and visualization** of healthcare inequalities in Maharashtra.


## Setup: Import Libraries for Selenium and Data Handling

We will use **Selenium** to scrape hospital data from the Jeevandayee portal and **pandas** to store the data in a CSV file.  

**Libraries used:**
- `selenium` – to control the web browser and interact with the portal.
- `pandas` – to store and manipulate the scraped data.
- `os` – to handle file paths.


In [ ]:
# Import Selenium modules for browser automation
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

# Import pandas for data handling
import pandas as pd

# Import os for file path operations
import os

## Initialize WebDriver and Wait

We set up the Chrome WebDriver to control the browser and interact with the portal.  

- `driver = webdriver.Chrome()` → Opens a Chrome browser window for automation.  
- `wait = WebDriverWait(driver, 25)` → Waits up to 25 seconds for elements to load before performing actions.  


In [172]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 25)

## Set Portal URL

We store the Jeevandayee Portal URL in a variable.  
This URL points to the **hospital information page** for Maharashtra.

- `hospital_info_url` → The link to the page where hospital data will be scraped.


In [18]:
hospital_info_url = "https://www.jeevandayee.gov.in/MJPJAY/FrontServlet?requestType=PublicViewsRH&actionVal=ViewBedInfoForDisease&City=x%20&Disease=-1&DataFlag=true&DfltHospList=Reports"

## Scrape Hospital Data from the Portal

Steps to get hospital data from the Jeevandayee portal:

1. Open the hospital info page in the browser.  
2. Click the **Get Information** button to load the table.  
3. Wait for the hospital table to appear.  
4. Get all table rows except the header and footer.  
5. Loop through each row and extract:
   - Hospital Name  
   - District  
   - Total Number of Beds  
6. Save the extracted data in a list of dictionaries (`hospital_data_list`) for later use.


In [ ]:
driver.get(hospital_info_url)  # Open the hospital info page

get_info_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@value='Get Information']")))
get_info_button.click()  # Click the "Get Information" button

hospital_table = wait.until(EC.presence_of_element_located((By.XPATH, "//table[@class='tabBorder']")))  # Wait for table
table_rows = hospital_table.find_elements(By.XPATH, ".//tr[position()>1 and position()<last()]")  # Skip header/footer

hospital_data_list = []  # Store scraped data

for row in table_rows:
    cells = row.find_elements(By.TAG_NAME, "td")
    if len(cells) >= 4:
        hospital_name = cells[1].text.strip()
        district = cells[2].text.strip()
        total_beds = cells[3].text.strip()
        
        hospital_data_list.append({
            "Hospital Name": hospital_name,
            "District": district,
            "Total Number of Beds": total_beds
        })

In [92]:
hospital_df = pd.DataFrame(hospital_data_list)
hospital_df

,Hospital Name,District,Total Number of Beds
0,INDRAVATI HOSPITAL & RESEARCH CENTRE,THANE,30
1,MAHATMA PHULE CHARITABLE TRUST HOSPITAL,THANE,100
2,REHBAR FOUNDATION MALIK RAKIYA KHATOON DIALYSI...,MUMBAI,15
3,SANJEEVANI HOSPITAL,NAGPUR,30
4,A & G HOSPITAL,THANE,50
...,...,...,...
2369,suryodaya hospital,NASHIK,50
2370,sushrut Hospital,DHARASHIV,30
2371,suvidha hospital and icu centre,DHARASHIV,50
2372,swara hospital,JALGAON,30


## Count Unique Hospital-District Combinations

Check how many **unique hospital and district pairs** are in the DataFrame.  
This helps verify if there are duplicate entries in the scraped data.


In [93]:
len(hospital_df[["Hospital Name", "District"]].drop_duplicates())

2370

## Remove Duplicate Hospital Entries

Remove duplicate rows based on **Hospital Name and District**.  
Keep the first occurrence and reset the index for a clean DataFrame.


In [94]:
hospital_df = hospital_df.drop_duplicates(subset=["Hospital Name", "District"], keep="first").reset_index(drop=True)

In [98]:
hospital_df

,Hospital Name,District,Total Number of Beds
0,INDRAVATI HOSPITAL & RESEARCH CENTRE,THANE,30
1,MAHATMA PHULE CHARITABLE TRUST HOSPITAL,THANE,100
2,REHBAR FOUNDATION MALIK RAKIYA KHATOON DIALYSI...,MUMBAI,15
3,SANJEEVANI HOSPITAL,NAGPUR,30
4,A & G HOSPITAL,THANE,50
...,...,...,...
2365,suryodaya hospital,NASHIK,50
2366,sushrut Hospital,DHARASHIV,30
2367,suvidha hospital and icu centre,DHARASHIV,50
2368,swara hospital,JALGAON,30


## Update Hospital Details with Additional Info

We now enrich the hospital DataFrame (`hospital_details_df`) with additional columns:  
- HOSP DISP CODE  
- Taluka  
- Address  
- Pincode  
- MCO Contact Number  

Steps:  
1. Navigate to the **Network Hospitals** section on the portal.  
2. Click **All** to view all hospitals.  
3. Scrape detailed information from the table.  
4. Match hospitals by name and district, and update the DataFrame.


In [ ]:
# Create a copy of the DataFrame and add new columns
hospital_details_df = hospital_df.copy()
hospital_details_df["HOSP DISP CODE"] = None
hospital_details_df["Taluka"] = None
hospital_details_df["Address"] = None
hospital_details_df["Pincode"] = None
hospital_details_df["MCO Contact Number"] = None

# Open the portal
driver.get(hospital_info_url)

# Navigate to Network Hospitals menu
network_menu = wait.until(EC.presence_of_element_located((By.LINK_TEXT, "Network Hospitals")))
webdriver.ActionChains(driver).move_to_element(network_menu).perform()
network_link = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@onclick='javascript:viewHosp()']")))
network_link.click()

# Click "All" link to view all hospitals
all_link = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@class='tableBluelnks' and text()='All']")))
driver.execute_script("arguments[0].scrollIntoView();", all_link)
driver.execute_script("arguments[0].click();", all_link)

# Wait for hospital table and get rows
hospital_table = wait.until(EC.presence_of_element_located((By.ID, "myTable")))
table_rows = hospital_table.find_elements(By.XPATH, ".//tr[position()>1 and position()<last()]")

updated = 0  # Counter for updated hospitals
for row in table_rows:
    cells = row.find_elements(By.TAG_NAME, "td")
    if len(cells) >= 10:
        hosp_name = cells[5].text.strip()
        district_name = cells[3].text.strip()
        hosp_disp_code = cells[2].text.strip() or None
        taluka = cells[4].text.strip() or None

        # Extract address from link if available
        try:
            address_link = cells[6].find_element(By.TAG_NAME, "a")
            address = address_link.get_attribute("href") or address_link.get_attribute("onclick")
            address = address.split("viewAddress('")[1].split("',")[0].strip()
        except:
            address = cells[6].text.strip() or None

        # Get pincode if available
        try:
            pincode = cells[7].text.strip() or None
        except IndexError:
            pincode = None

        # Get MCO contact number if available
        try:
            mco_contact = cells[8].text.strip() or None
        except IndexError:
            mco_contact = None

        # Match hospital by name and district, then update DataFrame
        mask = (
            (hospital_details_df["Hospital Name"].str.strip() == hosp_name) &
            (hospital_details_df["District"].str.strip() == district_name)
        )
        if mask.any():
            hospital_details_df.loc[mask, "HOSP DISP CODE"] = hosp_disp_code
            hospital_details_df.loc[mask, "Taluka"] = taluka
            hospital_details_df.loc[mask, "Address"] = address
            hospital_details_df.loc[mask, "Pincode"] = pincode
            hospital_details_df.loc[mask, "MCO Contact Number"] = mco_contact
            updated += 1

# Print total number of hospitals updated
print(f"Hospitals updated: {updated}")

Hospitals updated: 2101


In [162]:
hospital_details_df

,Hospital Name,District,Total Number of Beds,HOSP DISP CODE,Taluka,Address,Pincode,MCO Contact Number
0,INDRAVATI HOSPITAL & RESEARCH CENTRE,THANE,30,IAAHC,NA,"Plot no. GHANSOLI VILLAGE, RCC HOUSE, ARJUNE W...",400701,8452862559
1,MAHATMA PHULE CHARITABLE TRUST HOSPITAL,THANE,100,MPCT,NA,"C7, Budhyadev Mandir Marg, Sector 4, Sanpada, ...",400705,9702693695
2,REHBAR FOUNDATION MALIK RAKIYA KHATOON DIALYSI...,MUMBAI,15,None,None,None,None,None
3,SANJEEVANI HOSPITAL,NAGPUR,30,SANEI,NA,"NEAR TAHSIL OFFICE, GOTMARE LAYOUT, BRAHMI KA...",441501,9860447504
4,A & G HOSPITAL,THANE,50,AGAHT,NA,Suchak House murbad road opp karnik road kalya...,421301,9326114930
...,...,...,...,...,...,...,...,...
2365,suryodaya hospital,NASHIK,50,SURY,Nashik,"2,near dwarka,renuka nagar,near dwarka,NASHIK",422001,7588194242
2366,sushrut Hospital,DHARASHIV,30,SURHH,NA,"Plot No. 27 / 1174, samarth nagar, waruda roa...",413501,9422069885
2367,suvidha hospital and icu centre,DHARASHIV,50,SHI,Osmanabad,"28/438,28/439,Dic Road ,Barshi Bypass,Near Hp ...",413501,9890922151
2368,swara hospital,JALGAON,30,SWAR,NA,"12B,behind brookebond coloney ,ringroad,near b...",425001,9922307744


## Add Hospital Type Information

We now update each hospital with its **type (Government or Corporate)**:

1. Copy the detailed hospital DataFrame.  
2. Navigate to the portal and select each hospital type from the dropdown.  
3. Click "Get Information" to load the hospital table.  
4. Match hospitals by name and district, then update the **Hospital Type** column.  
5. Print how many hospitals were updated for each type.


In [ ]:
# Define hospital types
hospital_types = {"G": "Government", "C": "Corporate"}

# Create a copy of the hospital DataFrame and add a new column for type
hospital_with_type_df = hospital_details_df.copy()
hospital_with_type_df["Hospital Type"] = None

# Open the portal
driver.get(hospital_info_url)

try:
    # Loop through each hospital type
    for type_value, type_name in hospital_types.items():
        try:
            # Locate hospital type dropdown
            hosp_type_dropdown = wait.until(EC.presence_of_element_located((By.NAME, "OptionHosp")))
        
            # Select the option matching the current type
            for option in hosp_type_dropdown.find_elements(By.TAG_NAME, "option"):
                if option.get_attribute("value") == type_value:
                    option.click()
                    break
            
            # Click "Get Information" button
            get_info_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@value='Get Information']")))
            get_info_button.click()
            
            # Wait for hospital table to load
            hospital_table = wait.until(EC.presence_of_element_located((By.XPATH, "//table[@class='tabBorder']")))
            table_rows = hospital_table.find_elements(By.XPATH, ".//tr[position()>1 and position()<last()]")
            
            updated_count = 0
            for row in table_rows:
                cells = row.find_elements(By.TAG_NAME, "td")                    
                if len(cells) >= 3:
                    hosp_name = cells[1].text.strip()
                    district = cells[2].text.strip()
                    # Match hospital by name and district
                    mask = (
                        (hospital_with_type_df["Hospital Name"].str.strip() == hosp_name) &
                        (hospital_with_type_df["District"].str.strip() == district)
                    )
                    if mask.any():
                        hospital_with_type_df.loc[mask, "Hospital Type"] = type_name
                        updated_count += mask.sum()
                        
            print(f"Type '{type_name}': {updated_count} hospitals updated.")
                        
        except:
            print(f"No hospitals found for hospital type: {type_name}. Continuing to next type.")

except:
        print("No hospital type dropdown found.")

Type 'Government': 656 hospitals updated.
Type 'Corporate': 1713 hospitals updated.


In [164]:
hospital_with_type_df

,Hospital Name,District,Total Number of Beds,HOSP DISP CODE,Taluka,Address,Pincode,MCO Contact Number,Hospital Type
0,INDRAVATI HOSPITAL & RESEARCH CENTRE,THANE,30,IAAHC,NA,"Plot no. GHANSOLI VILLAGE, RCC HOUSE, ARJUNE W...",400701,8452862559,Corporate
1,MAHATMA PHULE CHARITABLE TRUST HOSPITAL,THANE,100,MPCT,NA,"C7, Budhyadev Mandir Marg, Sector 4, Sanpada, ...",400705,9702693695,Corporate
2,REHBAR FOUNDATION MALIK RAKIYA KHATOON DIALYSI...,MUMBAI,15,None,None,None,None,None,Corporate
3,SANJEEVANI HOSPITAL,NAGPUR,30,SANEI,NA,"NEAR TAHSIL OFFICE, GOTMARE LAYOUT, BRAHMI KA...",441501,9860447504,Corporate
4,A & G HOSPITAL,THANE,50,AGAHT,NA,Suchak House murbad road opp karnik road kalya...,421301,9326114930,Corporate
...,...,...,...,...,...,...,...,...,...
2365,suryodaya hospital,NASHIK,50,SURY,Nashik,"2,near dwarka,renuka nagar,near dwarka,NASHIK",422001,7588194242,Corporate
2366,sushrut Hospital,DHARASHIV,30,SURHH,NA,"Plot No. 27 / 1174, samarth nagar, waruda roa...",413501,9422069885,Corporate
2367,suvidha hospital and icu centre,DHARASHIV,50,SHI,Osmanabad,"28/438,28/439,Dic Road ,Barshi Bypass,Near Hp ...",413501,9890922151,Corporate
2368,swara hospital,JALGAON,30,SWAR,NA,"12B,behind brookebond coloney ,ringroad,near b...",425001,9922307744,Corporate


## Add Government Sub-Type for Hospitals

We now update **Government hospitals** with their specific **sub-type**:

1. Copy the hospital DataFrame that already has types.  
2. Select "Government" in the hospital type dropdown.  
3. Check if a Government sub-type dropdown exists.  
4. Loop through each sub-type and scrape hospital info.  
5. Match hospitals by name, district, and type, then update the **Government Sub-Type** column.  
6. Print how many hospitals were updated for each sub-type.


In [ ]:
# Create a copy of the DataFrame and add a new column for Government Sub-Type
hospital_with_govt_subtype_df = hospital_with_type_df.copy()
hospital_with_govt_subtype_df["Government Sub-Type"] = None

# Open the portal
driver.get(hospital_info_url)

# Select "Government" in hospital type dropdown
hosp_type_dropdown = wait.until(EC.presence_of_element_located((By.NAME, "OptionHosp")))
for option in hosp_type_dropdown.find_elements(By.TAG_NAME, "option"):
    if option.get_attribute("value") == "G":
        option.click()
        break

try:
    # Locate Government sub-type dropdown
    govt_dropdown = wait.until(EC.presence_of_element_located((By.NAME, "OptionGovtHosp")))
    select_govt = Select(govt_dropdown)
    govt_options = [opt.text.strip() for opt in select_govt.options if opt.get_attribute("value") != "-1"]

    # Loop through each Government sub-type
    for sub_type in govt_options:
        try:
            govt_dropdown = wait.until(EC.presence_of_element_located((By.NAME, "OptionGovtHosp")))
            select_govt = Select(govt_dropdown)
            select_govt.select_by_visible_text(sub_type)

            get_info_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@value='Get Information']")))
            get_info_button.click()

            # Wait for hospital table to load
            hospital_table = wait.until(EC.presence_of_element_located((By.XPATH, "//table[@class='tabBorder']")))
            table_rows = hospital_table.find_elements(By.XPATH, ".//tr[position()>1 and position()<last()]")

            updated_count = 0
            for row in table_rows:
                cells = row.find_elements(By.TAG_NAME, "td")
                if len(cells) >= 3:
                    hosp_name = cells[1].text.strip()
                    district = cells[2].text.strip()
                    # Match hospital by name, district, and type
                    mask = (
                        (hospital_with_govt_subtype_df["Hospital Name"].str.strip() == hosp_name) &
                        (hospital_with_govt_subtype_df["District"].str.strip() == district) &
                        (hospital_with_govt_subtype_df["Hospital Type"] == "Government")
                    )
                    if mask.any():
                        hospital_with_govt_subtype_df.loc[mask, "Government Sub-Type"] = sub_type
                        updated_count += mask.sum()

            print(f"Government sub-type '{sub_type}': {updated_count} hospitals updated.")

        except:
            print(f"No hospitals found for Government sub-type: {sub_type}. Continuing to next sub-type.")

except:
    print("No Government sub-type dropdown found or no hospitals for Government type.")

Government sub-type 'DH': 0 hospitals updated.
Government sub-type 'DHS': 35 hospitals updated.
Government sub-type 'DMER': 23 hospitals updated.
Government sub-type 'GH': 1 hospitals updated.
Government sub-type 'MCGM': 15 hospitals updated.
Government sub-type 'NMMC': 0 hospitals updated.
Government sub-type 'SDH100': 1 hospitals updated.
Government sub-type 'WH': 0 hospitals updated.


## Finalize Hospital DataFrame

We now clean and prepare the hospital DataFrame for storage:

1. Copy the DataFrame with Government sub-types.  
2. Rename columns to match the **database schema**.  
3. Strip whitespace from string columns.  
4. Remove duplicate hospital-district entries.  
5. Add a unique `hospital_id` as the index.  
6. Reorder columns and set `hospital_id` as the index.  
7. Display the first few rows to verify.


In [ ]:
# Copy the DataFrame
hospital_df_final = hospital_with_govt_subtype_df.copy()

# Rename columns to match database schema
hospital_df_final.rename(columns={
    "Hospital Name": "hospital_name",
    "District": "district",
    "Total Number of Beds": "total_beds",
    "HOSP DISP CODE": "hosp_disp_code",
    "Taluka": "taluka",
    "Address": "address",
    "Pincode": "pincode",
    "MCO Contact Number": "mco_contact_number",
    "Hospital Type": "hospital_type",
    "Government Sub-Type": "government_sub_type"
}, inplace=True)

# Strip whitespace from string columns
str_cols = ["hospital_name", "district", "hospital_type", "government_sub_type", "taluka", "address", "mco_contact_number", "hosp_disp_code"]
for col in str_cols:
    hospital_df_final[col] = hospital_df_final[col].astype(str).str.strip()

# Remove duplicate hospital-district entries
hospital_df_final = hospital_df_final.drop_duplicates(subset=["hospital_name", "district"]).reset_index(drop=True)

# Add unique hospital_id
hospital_df_final["hospital_id"] = hospital_df_final.index + 1

# Reorder columns and set hospital_id as index
column_order = ["hosp_disp_code","hospital_name","address","taluka","district","pincode","mco_contact_number","total_beds","hospital_type","government_sub_type"]
hospital_df_final = hospital_df_final[column_order]
hospital_df_final.set_index("hospital_id", inplace=True)

# Display first few rows
hospital_df_final.head()

,hosp_disp_code,hospital_name,address,taluka,district,pincode,mco_contact_number,total_beds,hospital_type,government_sub_type
hospital_id,,,,,,,,,,
1,IAAHC,INDRAVATI HOSPITAL & RESEARCH CENTRE,"Plot no. GHANSOLI VILLAGE, RCC HOUSE, ARJUNE W...",NA,THANE,400701,8452862559,30,Corporate,None
2,MPCT,MAHATMA PHULE CHARITABLE TRUST HOSPITAL,"C7, Budhyadev Mandir Marg, Sector 4, Sanpada, ...",NA,THANE,400705,9702693695,100,Corporate,None
3,None,REHBAR FOUNDATION MALIK RAKIYA KHATOON DIALYSI...,None,None,MUMBAI,None,None,15,Corporate,None
4,SANEI,SANJEEVANI HOSPITAL,"NEAR TAHSIL OFFICE, GOTMARE LAYOUT, BRAHMI KA...",NA,NAGPUR,441501,9860447504,30,Corporate,None
5,AGAHT,A & G HOSPITAL,Suchak House murbad road opp karnik road kalya...,NA,THANE,421301,9326114930,50,Corporate,None


In [190]:
hospital_df_final.to_csv("../data/maharashtra/hospital.csv", index=True)

## Scrape Hospital Service Categories

We now extract all **hospital service categories** from the portal:

1. Open the hospital info page.  
2. Locate the category (disease/service) dropdown.  
3. Get all options except the placeholder (`value=-1`).  
4. Create a DataFrame with `category_id` and `category_name`.  
5. Set `category_id` as the index.


In [ ]:
# Open the portal
driver.get(hospital_info_url)

# Locate the category dropdown and create a Select object
category_dropdown = wait.until(EC.presence_of_element_located((By.ID, "DiseaseM")))
select_category = Select(category_dropdown)

# Get all category names except the placeholder
category_options = [opt.text.strip() for opt in select_category.options if opt.get_attribute("value") != "-1"]

# Create a DataFrame for categories
category_df = pd.DataFrame({
    "category_id": range(1, len(category_options)+1),
    "category_name": category_options
}).set_index("category_id")

# Display the DataFrame
category_df

,category_name
category_id,
1,Burns (S13)
2,Cardiac And Cardiothoracic Surgery (S7)
3,Cardiology (M7)
4,Critical Care (M3)
5,Dermatology (M11)
6,ENT (S2)
7,Endocrinology (M13)
8,General Medicine (M16)
9,General Surgery (S1)


In [33]:
category_df.to_csv("../data/maharashtra/category.csv", index=True)

In [34]:
hospital_df_final = pd.read_csv("../data/maharashtra/hospital.csv", index_col=0)
category_df = pd.read_csv("../data/maharashtra/category.csv", index_col=0)

## Create Lookup Dictionaries

We create **lookup dictionaries** to quickly map:

- `(hospital_name, district)` → `hospital_id`  
- `category_name` → `category_id`  

These will help when creating the **hospital-category mapping table**.


In [ ]:
# Create a lookup for hospitals by (name, district)
hospital_lookup = {
    (row["hospital_name"].strip().lower(), row["district"].strip().lower()): hid
    for hid, row in hospital_df_final.iterrows()
}

# Create a lookup for categories by name
category_lookup = {
    row["category_name"].strip().lower(): cid
    for cid, row in category_df.iterrows()
}

## Initialize Hospital-Category Mapping

We prepare to create the **hospital-category mapping CSV**:

- `last_success_index` → Tracks the last successfully processed row (useful for resuming).  
- `file_path` → Path to save the mapping CSV.


In [45]:
last_success_index = -1
file_path = "../data/maharashtra/hospital_category.csv"

## Scrape Hospital-Category Mapping

We now create the **hospital-category mapping table**:

1. Loop through each service category.  
2. Select the category on the portal and click "Get Information".  
3. Extract hospital name, district, and last update date.  
4. If the hospital and category exist in the lookup dictionaries, add a mapping record.  
5. Append or create the `hospital_category.csv` file for each category.  
6. Track the `last_success_index` to resume in case of errors.


In [ ]:
# Open the portal
driver.get(hospital_info_url)

# Get all category names
category_dropdown = wait.until(EC.presence_of_element_located((By.ID, "DiseaseM")))
select_category = Select(category_dropdown)
category_texts = [str(opt.text).strip() for opt in select_category.options if opt.get_attribute("value") != "-1"]

# Loop through categories starting from last success
for idx in range(last_success_index + 1, len(category_texts)):
    hospital_category_records = []
    cat_text = category_texts[idx]
    try:
        # Select current category
        category_dropdown = wait.until(EC.presence_of_element_located((By.ID, "DiseaseM")))
        select_category = Select(category_dropdown)
        select_category.select_by_visible_text(cat_text)

        # Click "Get Information" button
        get_info_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@value='Get Information']")))
        get_info_button.click()

        # Wait for hospital table and get rows
        hospital_table = wait.until(EC.presence_of_element_located((By.XPATH, "//table[@class='tabBorder']")))
        table_rows = hospital_table.find_elements(By.XPATH, ".//tr[position()>1 and position()<last()]")

        valid_count = 0
        for row in table_rows:
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) >= 7:
                hosp_name = cells[1].text.strip()
                district = cells[2].text.strip()
                last_update = cells[6].text.strip()

                # Only include if last update exists
                if last_update != "--":
                    key = (hosp_name.lower(), district.lower())
                    hid = hospital_lookup.get(key)
                    cid = category_lookup.get(cat_text.strip().lower())
                    if hid is not None and cid is not None:
                        hospital_category_records.append({
                            "hospital_id": hid,
                            "category_id": cid
                        })
                        valid_count += 1

        print(f"[{idx}] Category '{cat_text}': {valid_count} hospitals found")

        # Save to CSV (append if exists, else create new)
        partial_df = pd.DataFrame(hospital_category_records)
        if os.path.exists(file_path):
            partial_df.to_csv(file_path, mode='a', header=False, index=False)
        else:
            partial_df.to_csv(file_path, mode='w', header=True, index=False)

        # Update last success index
        last_success_index = idx

    except Exception as e:
        print(f"Error processing category '{cat_text}' at index {idx}: {e}") 
        break

[18] Category 'Mental Health Packages (M17)': 22 hospitals found
[19] Category 'Neonatal and Pediatric Medical Management (M6)': 376 hospitals found
[20] Category 'Nephrology (M8)': 742 hospitals found
[21] Category 'Neurology (M9)': 184 hospitals found
[22] Category 'Neurosurgery (S10)': 261 hospitals found
[23] Category 'OTHERS (OTH)': 0 hospitals found
[24] Category 'Ophthalmology Surgery (S3)': 142 hospitals found
[25] Category 'Orthopedic Surgery And Procedures (S5)': 725 hospitals found
[26] Category 'Pediatric Cancer (S21)': 2 hospitals found
[27] Category 'Pediatric Surgery (S8)': 86 hospitals found
[28] Category 'Physiotherapy (M20)': 0 hospitals found
[29] Category 'Plastic Surgery (S12)': 68 hospitals found
[30] Category 'Polytrauma (S14)': 198 hospitals found
[31] Category 'Prosthesis and Orthosis (S15)': 2 hospitals found
[32] Category 'Pulmonology (M10)': 332 hospitals found
[33] Category 'Radiation Oncology (M2)': 65 hospitals found
[34] Category 'Rheumatology (M12)': 19